In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import statistics 
from statistics import mode 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from xgboost import XGBClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import StratifiedKFold
import seaborn as sns 
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.DataFrame(pd.read_csv("/train.csv"))
test = pd.DataFrame(pd.read_csv("/test.csv"))
exp = pd.DataFrame(pd.read_csv("/monthly_expenditures.csv"))
exp3 = pd.DataFrame(pd.read_csv("/İŞML#3/expv2_b.csv"))
exp4 = pd.DataFrame(pd.read_csv("/İŞML#3/expv2_b2.csv"))
exp5 = pd.DataFrame(pd.read_csv("/İŞML#3/yuzde_v1.csv"))

nntr = pd.DataFrame(pd.read_csv("/İŞML#3/nn_train_150i_v3.csv"))
nnts = pd.DataFrame(pd.read_csv("/İŞML#3/nn_test_150i_v3.csv"))

def refresh_sub():
    sub = pd.read_csv('/sample_submission.csv')
    sub = pd.DataFrame(sub)
    sub = sub.drop(columns = 'target')
    return sub
def submit(prediction, name):
    sub = refresh_sub()
    sub.insert(1, 'target',prediction)
    sub.to_csv(name , index = False, header = True)

*********
DATA EXPLORATION
********

In [ ]:
sg=exp.groupby("sektor").sum()
fig, ax = plt.subplots(figsize=(25,7))
plt.bar(sg.index, sg.aylik_toplam_tutar)
plt.show()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(19,6))
#train.yas.hist(bins=30)
np.log(train.loc[train.kidem_suresi > 0]["kidem_suresi"] ).hist(bins=30)
#train.egitim.hist()
#train.is_durumu.hist()
#train.meslek_grubu.hist()
#train.target.hist()

In [ ]:
print(train.loc[train.target == 0].shape[0] / (train.shape[0]) )

In [ ]:
X_train.loc[train.target == 0].yas.hist(bins=32)

In [ ]:
X_train.loc[train.target == 1].yas.hist(bins=32)

In [ ]:
fig, ax = plt.subplots(figsize=(30,13))
plt.scatter(train.loc[train.target==0].yas, train.loc[train.target==0].yas - train.loc[train.target==0].kidem_suresi/12,c="b" ,s = 2)
plt.scatter(train.loc[train.target==1].yas, train.loc[train.target==1].yas - train.loc[train.target==1].kidem_suresi/12,c="R", s = 2)
plt.show()

In [ ]:
train.yas - train.kidem_suresi/52

In [ ]:
exp5.loc[exp5.musteri == "ffffdfc0f2"]

In [ ]:
disp_sum.iloc[-1].sum()

******
DATA PREPROCESSING
*******

In [ ]:
#dont run
if True:
    train_1 = train.copy();  test_1 = test.copy()
    train_1 = train_1.drop(columns = "tarih");  test_1 = test_1.drop(columns = "tarih")
    def tri_imputer(t, df, f2i, f):
        print(f2i, " is being imputed..." )
        [f1, f2] =  [f[j] for j in range(2)]
        empty_df = t.loc[t[f2i] != t[f2i] ]
        for i in range(empty_df.shape[0]):
            mus = empty_df.iloc[i].musteri;    f1val = empty_df.iloc[i][f1];    f2val = empty_df.iloc[i][f2]
            if f1val == f1val:
                if f2val == f2val:
                    try: df.loc[df.musteri == mus, f2i] = df.loc[(df[f1] == f1val) & (df[f2]== f2val) & (df[f2i] == df[f2i])][f2i].mode()[0]
                    except: 
                        if df.loc[df[f1] == f1val].shape[0] > df.loc[df[f2] == f2val].shape[0]:
                            try: df.loc[df.musteri == mus, f2i] = df.loc[(df[f1] == f1val) & (df[f2]== df[f2]) & (df[f2i] == df[f2i])][f2i].mode()[0]
                            except: print("skipped11")
                        else:
                            try: df.loc[df.musteri == mus, f2i] = df.loc[(df[f1] == df[f1]) & (df[f2]== f2val) & (df[f2i] == df[f2i])][f2i].mode()[0]
                            except: print("skipped12")
                else:
                    try: df.loc[df.musteri == mus, f2i] = df.loc[(df[f1] == f1val) & (df[f2]== df[f2]) & (df[f2i] == df[f2i])][f2i].mode()[0]
                    except: print("skipped2")
            else:
                if f2val == f2val:
                    try: df.loc[df.musteri == mus, f2i] = df.loc[(df[f1] == df[f1]) & (df[f2]== f2val) & (df[f2i] == df[f2i])][f2i].mode()[0]
                    except: print("skipped3")
                else:
                    try: df.loc[df.musteri == mus, f2i] = df.loc[(df[f1] == df[f1]) & (df[f2]== df[f2]) & (df[f2i] == df[f2i])][f2i].mode()[0]
                    except: print("skipped4")
        return df
    if True:
        tri_list = ["egitim", "is_durumu", "meslek_grubu"]
        for i in range(0,3):
            feature_to_impute = tri_list[i]
            supporting_features = tri_list[:i] + tri_list[i+1:]
            train_1 = tri_imputer(train, train_1, feature_to_impute, supporting_features)
            test_1 = tri_imputer(test, test_1, feature_to_impute, supporting_features)
        train_1.loc[train.kidem_suresi < 0, "kidem_suresi"] = train_1.loc[train.kidem_suresi > 1, "kidem_suresi"].mean()

In [ ]:
train_1 = train.copy();  test_1 = test.copy()
train_1 = train_1.drop(columns = "tarih");  test_1 = test_1.drop(columns = "tarih")
def bi_imputer(t, df, f2i, f):
    print(f2i, "is being imputed..." )
    empty_df = t.loc[t[f2i] != t[f2i] ]
    for i in range(empty_df.shape[0]):
        mus = empty_df.iloc[i].musteri;    fval = empty_df.iloc[i][f]
        if fval == fval:
            df.loc[df.musteri == mus, f2i] = df.loc[(df[f] == fval) & (df[f2i] == df[f2i])][f2i].mode()[0]
        else:
            df.loc[df.musteri == mus, f2i] = df.loc[(df[f] == df[f]) & (df[f2i] == df[f2i])][f2i].mode()[0]
    return df
bi_list = ["egitim", "is_durumu"]
for i in range(0,2):
    train_1 = bi_imputer(train, train_1, bi_list[i], bi_list[1-i])
    test_1  = bi_imputer(test,  test_1,  bi_list[i], bi_list[1-i])
train_1.loc[train_1.kidem_suresi < 0, "kidem_suresi"] = train_1.loc[train.kidem_suresi >= 0, "kidem_suresi"].mean()
test_1.loc[test_1.kidem_suresi < 0, "kidem_suresi"] = test_1.loc[train.kidem_suresi >= 0, "kidem_suresi"].mean()
print("Imputation completed.")
totdf = train_1.append(test_1, ignore_index = True).drop(["target"], axis="columns")
curmg = totdf.meslek_grubu
totdf= pd.get_dummies(totdf, columns = ["egitim", "is_durumu"] )
mgtestx  = totdf.loc[ totdf.meslek_grubu.isna() ].drop(["meslek_grubu"], axis ="columns")
mgtrainx = totdf.loc[~totdf.meslek_grubu.isna() ]
mgtrainx2= pd.get_dummies(mgtrainx, columns = ["meslek_grubu"] )
totdf= mgtrainx2.append(mgtestx, ignore_index = False)
totdf = totdf.sort_index()
#totdf[str(colname + "_x")] = totdf[str(colname + "_x")].fillna(totdf[str(colname + "_y")])
#totdf = totdf.drop(str(colname + "_y"), axis = "columns")
mgtrainx = mgtrainx.drop(["meslek_grubu", "musteri"], axis ="columns")
mgtestx = mgtestx.drop(["musteri"], axis ="columns")
#mgtrainx = pd.get_dummies(mgtrainx, columns = ["egitim", "is_durumu"])
#mgtestx = pd.get_dummies(mgtestx, columns = ["egitim", "is_durumu"])
totdf = totdf.set_index("musteri")
totdf["musteri"]=totdf.index
totdf["meslek_grubu"] = list(curmg)
customers_to_predict = totdf.loc[ totdf.meslek_grubu.isna() ].musteri
#totdf = totdf.drop(columns= "musteri")
for mg in totdf.meslek_grubu.unique():
    rf = RandomForestClassifier(random_state = 0)
    colname = "meslek_grubu_" + str(mg)
    if colname != "meslek_grubu_nan":
        print(colname, "is being predicted.")
        mgtrainy = totdf.loc[~totdf.meslek_grubu.isna(), colname ]
        rf.fit(mgtrainx, mgtrainy)
        predictions = rf.predict_proba(mgtestx)
        prediction_df = pd.DataFrame({ "musteri": customers_to_predict, colname: predictions[:,1]})
        totdf = totdf.drop(columns= "musteri")
        totdf = totdf.merge(prediction_df, how="left", left_index=True, right_index=True, copy=False)
        #totdf = totdf.merge(prediction_df, how="outer")
        #totdf[str(colname + "_x")] = totdf[str(colname + "_x")].fillna(totdf[str(colname + "_y")])
        totdf["musteri"] = totdf.index
        totdf.loc[totdf.meslek_grubu != totdf.meslek_grubu, colname + "_x"] = totdf.loc[totdf.meslek_grubu != totdf.meslek_grubu][colname + "_y"]
        totdf = totdf.drop(columns = colname + "_y").rename(columns = {colname + "_x": colname})
        #totdf = totdf.append(prediction_df).sort_values(colname).drop_duplicates(subset = "musteri")
totdf = totdf.drop(columns = "meslek_grubu")
totdf.index = totdf.musteri
train_1 = pd.merge(train_1, totdf.loc[:,	"meslek_grubu_070e3be3ae":], left_on="musteri", right_index=True, how = "inner")
test_1  = pd.merge(test_1, totdf.loc[:,	"meslek_grubu_070e3be3ae":], left_on="musteri", right_index=True, how = "inner")
train_1 = train_1.drop(columns = ["musteri_x","musteri_y",  "meslek_grubu"])
test_1  = test_1.drop(columns =  ["musteri_x","musteri_y",  "meslek_grubu"])
print("Predictions completed.")

In [ ]:
def correlation_heatmap(train, s):
    correlations = train.corr()
    fig, ax = plt.subplots(figsize=(s,s))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show();
    
correlation_heatmap(train_1[train_1.columns], 3)

In [ ]:
def month(s):    return str(s)[-4:-2]
exp.tarih = exp.tarih.apply(month)
for m in ["0" + str(i) for i in range(1,7)]:
    def add_month(s):    return str(s)+"_" + m
    subexp = exp.loc[exp.tarih == m]
    subexp.sektor = subexp.sektor.apply(add_month)
    expg = subexp.groupby(['musteri', "sektor"], as_index = False).sum()
    if m == "01":
        expv = expg.pivot_table(index = ["musteri"], columns="sektor", values = ["aylik_toplam_tutar"], aggfunc="sum", fill_value=0)
    else:
        expvnext = expg.pivot_table(index = ["musteri"], columns="sektor", values = ["aylik_toplam_tutar"], aggfunc="sum", fill_value=0)
        expv = expv.merge(expvnext, on ="musteri", how="outer")
expv =  expv.merge(pd.DataFrame(exp.musteri.unique(), columns=["musteri"]), on = "musteri", how="outer")
expv = expv.fillna(0)

In [ ]:
correlation_heatmap(expv[expv.columns], 30)

In [ ]:
#dontrun
if False:
    sektorler = exp.sektor.unique()
    sectors=[]
    expvtemp=expv.copy()
    for i in range(len(sektorler)):
        sector =[col for col in expv.columns];    j=0
        for k in range(len(sector)):
            k-=j
            if not str(sector[k][1]).startswith(sektorler[i]): sector.remove(sector[k]); j+=1
        sectors.append(sector)
        #print(sector)
        colname= "sum_exp_" + str(sektorler[i])[0:7]
        expvtemp[colname] = expvtemp[sector[0:12]].sum(axis=1)

In [ ]:
# weighting
if True:
    sektorler = exp.sektor.unique()
    sectors=[]
    expv2=expv.copy()
    for i in range(len(sektorler)):
        sector =[col for col in expv.columns];    j=0
        for k in range(len(sector)):
            k-=j
            if not str(sector[k][1]).startswith(sektorler[i]): sector.remove(sector[k]); j+=1
        sectors.append(sector)
        #print(sector)
        colname= "sum_exp_" + str(sektorler[i])[0:7] + "_weighted"
        expv2[colname] = np.dot(expv2[sector[0:12]], [i+1 for i in range(6)])
    expv2["all_weighted"] = expv2.iloc[:,79: ].mean(axis=1)

In [ ]:
correlation_heatmap(expv[expv.columns], 32)

*******
FEATURE ENGINEERING
********

In [ ]:
train_1["kayit_zamani"] = train_1.yas - train_1.kidem_suresi/12
test_1["kayit_zamani"] = test_1.yas - test_1.kidem_suresi/12

In [ ]:
subexp3 = exp3.loc[1:,["musteri","degisim" ,"degisim.1","degisim.2","degisim.3","degisim.4","degisim.5","degisim.6","degisim.7","degisim.8","degisim.9","degisim.10","degisim.11","degisim.12"]]
subexp3 = subexp3.astype({'degisim': 'float64', 'degisim.1': 'float64',  'degisim.2': 'float64', 'degisim.3': 'float64', 'degisim.4': 'float64', 'degisim.5': 'float64'
                        , 'degisim.6': 'float64', 'degisim.7': 'float64', 'degisim.8': 'float64', 'degisim.9': 'float64', 'degisim.10': 'float64', 'degisim.11': 'float64', 'degisim.12': 'float64'  })
try: expv3 = expv2.copy()
except: expv3 = expv.copy()
expv3 = expv3.merge(subexp3, on= "musteri", how="inner")
expv3 = expv3.drop(columns=("musteri", ""))

In [ ]:
correlation_heatmap(subexp3[subexp3.columns], 20)

In [ ]:
#dontrun
if False:
    def month(s):    return str(s)[-4:-2]
    exp.tarih = exp.tarih.apply(month)
    m_sum = exp.groupby(["musteri", "tarih"]).sum().pivot_table(index="musteri", columns="tarih", values="aylik_toplam_tutar", aggfunc="sum", fill_value=0)
    #m_sum_trunc = m_sum[["01", "05", "06"]]
    expv3 = expv3.merge(m_sum, on="musteri", how="inner")
    #expv3 = expv3.drop(columns= ("musteri", ""))

In [ ]:
correlation_heatmap(m_sum[m_sum.columns], 12)

In [ ]:
correlation_heatmap(expv3[expv3.columns], 32)

In [ ]:
from scipy.stats import linregress
def trend(x):    return linregress([i for i in range(6)], x).slope
disp_sum = exp.groupby(["musteri", "tarih", "sektor"]).sum().pivot_table(index="musteri", columns=["tarih", "sektor"], values="aylik_toplam_tutar", aggfunc="sum", fill_value=0)
for i in range (0,13):
    if i not in [99]:
        st = str(i) + "_Jan_Jun_%chg"
        sub_disp = disp_sum.iloc[:,0+i::13]
        prc = (sub_disp.iloc[:,5]-sub_disp.iloc[:,0] ) / (sub_disp.iloc[:,0] + 1)
        expv3[st] = list(prc)
    ###
    if i not in [99]:
        st = str(i) + "_trend"
        sub_disp = disp_sum.iloc[:,0+i::13]
        slope = list(sub_disp.apply(trend, axis=1))
        expv3[st] = slope
    ##
    if i not in [99]:
        st = str(i) + "_May_Jun_%chg"
        sub_disp = disp_sum.iloc[:,0+i::13]
        prc = (sub_disp.iloc[:,5]-sub_disp.iloc[:,4] ) / (sub_disp.iloc[:,4] +1)
        expv3[st] = list(prc)
del sub_disp, prc, st, slope

In [ ]:
correlation_heatmap(expv3[expv3.columns], 64)

In [ ]:
correlation_heatmap(exp5[exp5.columns], 36)

In [ ]:
#expv = expg.pivot_table(index = ["musteri"], columns="sektor", values = ["aylik_toplam_tutar", "islem_adedi"], aggfunc="sum", fill_value=0)
#exp5 = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/IS_ML/yuzde_v1.csv"))
exp5 = exp5.pivot_table(index = ["musteri"], columns=["sektor", "ay"], values = "yuzde" , aggfunc="mean", fill_value=0, dropna=False)
exp5.musteri = exp5.index
########exp5trunc = exp5.iloc[:,4::6].join(exp5.iloc[:,5::6])
#exp5 = exp5.add_suffix("_%_of_month")
#######expv3 = expv3.merge(exp5trunc, on="musteri", how="inner")
expv3 = expv3.merge(exp5, on="musteri", how="inner")

In [ ]:
correlation_heatmap(expv3[expv3.columns], 42)

In [ ]:
correlation_heatmap(exp6v[exp6v.columns], 36)

In [ ]:
exp6 = pd.DataFrame(pd.read_csv("/expb3-56.csv"))
exp6v = exp6.pivot_table(index = ["musteri"], columns=["sektor"], values = "islem_adedi_degisim_56" , aggfunc="mean", fill_value=0, dropna=False)
exp6v = exp6v.add_suffix("_adet_may_june")
exp6v.musteri = exp6v.index
expv3 = expv3.merge(exp6v, on="musteri", how="inner")

In [ ]:
correlation_heatmap(expv3[expv3.columns], 64)

In [ ]:
correlation_heatmap(exp7_bin[exp7_bin.columns], 40)

In [ ]:
exp7_bin = pd.DataFrame(pd.read_csv("/tutar_binary.csv"))
exp7_bin = exp7_bin.pivot_table(index = ["musteri"], columns=["sektor"], values = "tutar_binary" , aggfunc="mean", fill_value=0, dropna=False)
exp7_bin = exp7_bin.add_suffix("_may_jun_bin")
exp7_bin.musteri = exp7_bin.index
expv3 = expv3.merge(exp7_bin, on="musteri", how="inner")

In [ ]:
correlation_heatmap(expv3[expv3.columns], 80)

In [ ]:
# ALTERNATIVE
expv3trunc = expv3.iloc[:,66:]
expv3trunc["musteri"] = expv3.musteri

In [ ]:
train2 = train_1.copy()
test2 = test_1.copy()
train2 = train2.merge(expv3, on="musteri", how= "inner")
test2 = test2.merge(expv3, on="musteri", how= "inner")

In [ ]:
X_train = train2.drop(['target', "musteri"], axis='columns')
y_train = train_1.target
X_test = test2.drop([ "musteri"], axis='columns')

X_train["NN_pred"] = nntr["0"]
X_test["NN_pred"] = nnts["target"]

*******
PREDICTION MODELS BELOW
*******

In [ ]:
counter = Counter(y_train)
estimate = counter[0] / counter[1]
print('Estimate: %.3f' % estimate)

In [ ]:
#X_train = train2.drop(['target', "musteri"], axis='columns')
#y_train = train_1.target
#column_trans = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore'),['egitim', 'is_durumu']),remainder='passthrough')
column_trans = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore'),['egitim', 'is_durumu']),remainder='passthrough')
for md in [3,4]:
    for csbt in [1]:
        for mcw in [1,2,3]:
            for lr in [0.1]:
                for n in [70]:
                    for spw in [8]:
                        for g in [0]:
                            xgb = XGBClassifier(gamma = g, max_depth = md,scale_pos_weight = spw, n_estimators= n, colsample_bytree=csbt, min_child_weight = mcw, learning_rate = lr, eval_metric= "auc")
                            pipe = make_pipeline(column_trans, xgb)
                            print("mcwght: ",mcw ," l_r: ", lr  , " n: ", n, " spw: ", spw," csbt: ", csbt, " max_d: ", md , " gamma:", g, " AUC: " ,cross_val_score(pipe, X_train, y_train, cv=5, scoring="roc_auc").mean())

In [ ]:
#X_test = test2.drop([ "musteri"], axis='columns')
pipe.fit(X_train, y_train)
pred_test = pipe.predict_proba(X_test)[:,1]
submit(pred_test, r'/submissions/xgb_with_nn_v3_full.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(19,40))
plt.barh(X_train.columns[0:150], xgb.feature_importances_[0:150])
plt.show()

In [ ]:
pipe.fit(X_train, y_train)
predtrain = pipe.predict_proba(X_train)
#predtrain = xgb.predict_proba(X_train)
from sklearn import metrics
from sklearn.metrics import roc_auc_score
# roc graph 
fpr, tpr, _ = metrics.roc_curve(y_train, predtrain[:,1])
roc_auc = metrics.auc(fpr, tpr)
fig, ax = plt.subplots(figsize=(8,8))
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC AUC: %0.3f' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC')
plt.legend(loc="lower right")
plt.show()

******
MODEL VALIDATION
*****

In [ ]:
X_train.columns = ['{}_{}'.format(x[0], x[1]) for x in X_train.columns]

In [ ]:
X_train.columns = ['{}_{}'.format(x[0], x[1]) for x in X_train.columns]
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns
#X_train = X_train.rename(columns='_'.join, inplace=True)

X_trainval, X_testval, y_trainval, y_testval = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
numeric_transformer = Pipeline(steps=[    ('scaler', "passthrough")])
categorical_transformer = Pipeline(steps=[    ('one_hot', OneHotEncoder(handle_unknown = 'ignore'))])

#numeric_transformer = "passthrough"
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipev = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',  XGBClassifier(scale_pos_weight = 10, n_estimators= 85))])
#X_trainval = X_trainval.replace([np.inf, -np.inf], np.nan)
#X_testval = X_testval.replace([np.inf, -np.inf], np.nan)
#X_trainval.fillna(0)
#X_testval.fillna(0)
model = pipev.fit(X_trainval, y_trainval)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot', OneHotEncoder(handle_unknown = 'ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',  XGBClassifier(scale_pos_weight = 10, n_estimators= 85))])

model = pipe.fit(X_trainval, y_trainval)

In [ ]:
from sklearn.metrics import classification_report
target_names = y_testval.unique().astype(str)
y_pred = model.predict(X_testval)
print(classification_report(y_testval, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94     11525
           1       0.14      0.32      0.19       475

    accuracy                           0.89     12000
   macro avg       0.55      0.62      0.57     12000
weighted avg       0.94      0.89      0.91     12000



In [ ]:
onehot_columns = list(pipev.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot'].get_feature_names(input_features=categorical_features))
numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)

In [ ]:
eli5.explain_weights(xgb, top=50, feature_names=numeric_features_list)

In [ ]:
Xp = preprocessor.fit_transform(X_trainval)

In [ ]:
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

continuous_transformer = 'passthrough'
categorical_columns = ["egitim", "is_durumu", "meslek_grubu"]

continuous_columns = [col for col in X_train.columns if col not in categorical_columns]
ct = ColumnTransformer(
    [           ('categorical', categorical_transformer, categorical_columns),
                ('continuous', continuous_transformer, continuous_columns),    ]
    ,
    sparse_threshold=0.,    n_jobs=-1)
#pipe[0].get_feature_names()

pipe2 = Pipeline(steps=[('preprocessor', ct),
                      ('clf', xgb)])

In [ ]:
pipe2.fit(X_train,y_train)

In [ ]:
for name,step in pipe2.named_steps.items():
    if hasattr(step, 'get_feature_names'):
        print(step.get_feature_names())

In [ ]:
pipe2['preprocessor'].transformers[1][1]['onehot']\
                   .get_feature_names(categorical_features)

In [ ]:
from xgboost import plot_importance
pd.DataFrame(pipe.steps[1][1].feature_importances_.reshape(1, -1), columns=X_train.columns)
#figure(num=None, figsize=(20, 14), dpi=80, facecolor='w', edgecolor='k')
ax = plot_importance(xgb)
fig = ax.figure
fig.set_size_inches(20, 8)
plt.show()